#Work as SQL

In [1]:
import pandas as pd 
import sqlite3

In [9]:
#Downloader
# https://stackoverflow.com/questions/22676/how-do-i-download-a-file-over-http-using-python
url = "https://sisa.msal.gov.ar/datos/descargas/covid-19/files/Covid19Casos.csv"


import os,requests
def download(url):
    get_response = requests.get(url,stream=True)
    file_name  = url.split("/")[-1]
    with open(file_name, 'wb') as f:
        for chunk in get_response.iter_content(chunk_size=1024):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
download(url)



In [ ]:
pip3 install wget

In [20]:
#Downloader
# https://medium.com/@petehouston/download-files-with-progress-in-python-96f14f6417a2
url = "https://sisa.msal.gov.ar/datos/descargas/covid-19/files/Covid19Casos.csv"

import wget
def download(url):
    def bar_custom(current, total, width=80):
        print("Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total))
    wget.download(url,'./test/',bar=bar_custom)

download(url)


8352 / 173302083] bytes
Downloading: 98% [169836544 / 173302083] bytes
Downloading: 98% [169844736 / 173302083] bytes
Downloading: 98% [169852928 / 173302083] bytes
Downloading: 98% [169861120 / 173302083] bytes
Downloading: 98% [169869312 / 173302083] bytes
Downloading: 98% [169877504 / 173302083] bytes
Downloading: 98% [169885696 / 173302083] bytes
Downloading: 98% [169893888 / 173302083] bytes
Downloading: 98% [169902080 / 173302083] bytes
Downloading: 98% [169910272 / 173302083] bytes
Downloading: 98% [169918464 / 173302083] bytes
Downloading: 98% [169926656 / 173302083] bytes
Downloading: 98% [169934848 / 173302083] bytes
Downloading: 98% [169943040 / 173302083] bytes
Downloading: 98% [169951232 / 173302083] bytes
Downloading: 98% [169959424 / 173302083] bytes
Downloading: 98% [169967616 / 173302083] bytes
Downloading: 98% [169975808 / 173302083] bytes
Downloading: 98% [169984000 / 173302083] bytes
Downloading: 98% [169992192 / 173302083] bytes
Downloading: 98% [170000384 / 173302

In [3]:
#read the CSV
df = pd.read_csv('./data/casoscovid19.csv')
#connect to a database
conn = sqlite3.connect("casoscovid19.db") #if the db does not exist, this creates a Any_Database_Name.db file in the current directory
#store your table in the database:


cursor = conn.cursor()
#Doping EMPLOYEE table if already exists
cursor.execute("DROP TABLE IF EXISTS test_0")
#print("Table dropped... ")
#Commit your changes in the database
conn.commit()
#Closing the connection
#conn.close()
# https://www.tutorialspoint.com/python_sqlite/python_sqlite_drop_table.htm

#gg = pd.read_sql('DROP TABLE IF EXISTS test_0;', conn)
#pd.read_sql_query('DROP TABLE IF EXISTS test_0;', conn)
df.to_sql('test_0', conn)

In [4]:
#connect to a database
conn = sqlite3.connect("casoscovid19.db") 
#if the db does not exist, this creates a Any_Database_Name.db file in the current directory

In [5]:
# table argentina
# Provincia	Casos Confirmados Totales	Recuperados	Fallecidos
sql_string = '''SELECT count(*) AS Total,
    sum(case when clasificacion_resumen="Confirmado" then 1 else 0 end) AS Confirmados,
    sum(case when clasificacion LIKE "%No Activo%" then 1 else 0 end) AS Recuperados,
    sum(case when clasificacion = "Caso confirmado - Fallecido" then 1 else 0 end) AS Fallecidos
FROM test_0;
'''
gg = pd.read_sql(sql_string, conn)
gg

,Total,Confirmados,Recuperados,Fallecidos
0,791878,235664,165065,4411


In [6]:
# table provincias
# Provincia	Casos Confirmados Totales	Recuperados	Fallecidos
sql_string = '''SELECT residencia_provincia_nombre  AS Provincia,
    count(*) AS Total,
    sum(case when clasificacion_resumen="Confirmado" then 1 else 0 end) AS Confirmados,
    sum(case when clasificacion LIKE "%No Activo%" then 1 else 0 end) AS Recuperados,
    sum(case when clasificacion = "Caso confirmado - Fallecido" then 1 else 0 end) AS Fallecidos
FROM test_0
GROUP BY residencia_provincia_nombre;
'''
gg = pd.read_sql(sql_string, conn)
gg

,Provincia,Total,Confirmados,Recuperados,Fallecidos
0,Buenos Aires,425166,143511,101006,2469
1,CABA,180552,67680,47859,1441
2,Catamarca,2241,60,54,0
3,Chaco,25739,3949,3312,165
4,Chubut,3539,310,252,3
5,Corrientes,4930,204,163,2
6,Córdoba,36495,3156,1861,61
7,Entre Ríos,5932,1056,737,12
8,Formosa,915,81,78,0
9,Jujuy,15526,3027,2222,31


In [7]:
# table departamentos buenos aires
# Provincia	Casos Confirmados Totales	Recuperados	Fallecidos
sql_string = '''SELECT residencia_departamento_nombre  AS Residencia,
    count(*) AS Total,
    sum(case when clasificacion_resumen="Confirmado" then 1 else 0 end) AS Confirmados,
    sum(case when clasificacion LIKE "%No Activo%" then 1 else 0 end) AS Recuperados,
    sum(case when clasificacion = "Caso confirmado - Fallecido" then 1 else 0 end) AS Fallecidos
FROM test_0
WHERE (residencia_provincia_nombre="Buenos Aires")
GROUP BY residencia_departamento_nombre;
'''
gg = pd.read_sql(sql_string, conn)
gg

,Residencia,Total,Confirmados,Recuperados,Fallecidos
0,25 de Mayo,204,37,28,0
1,9 de Julio,293,21,15,1
2,Adolfo Alsina,36,3,2,0
3,Adolfo Gonzales Chaves,36,1,0,0
4,Alberti,162,13,9,0
...,...,...,...,...,...
131,Tres de Febrero,14131,5235,3799,110
132,Vicente López,8217,2457,1595,61
133,Villa Gesell,169,9,8,0
134,Villarino,131,19,19,0


In [8]:
# table departamentos buenos aires
# Provincia	Casos Confirmados Totales	Recuperados	Fallecidos
sql_string = '''
SELECT fecha_apertura, count(*) FROM test_0 WHERE (clasificacion_resumen="Confirmado") group by fecha_apertura  ORDER BY "fecha_apertura" DESC LIMIT 7;
'''
gg = pd.read_sql(sql_string, conn)
gg

,fecha_apertura,count(*)
0,2020-08-07,1400
1,2020-08-06,4044
2,2020-08-05,4782
3,2020-08-04,6464
4,2020-08-03,5979
5,2020-08-02,4017
6,2020-08-01,4029
